# Try Alex Goodman's approach for efficient access to references
* https://discourse.pangeo.io/t/trick-for-improving-kerchunk-performance-for-large-numbers-of-chunks-files/3090
* https://github.com/fsspec/kerchunk/issues/293#issuecomment-1416290468

In [1]:
import fsspec
import xarray as xr

In [4]:
%run parquet_refs.py

In [2]:
fs = fsspec.filesystem('s3', anon=True, 
                        client_kwargs={'endpoint_url':'https://ncsa.osn.xsede.org'})

In [ ]:
url = 's3://esip/noaa/nwm/grid1km/LDAS_combined.json'

In [ ]:
fs.size(url)/1e9 # GB

In [ ]:
%%time
refs = ujson.load(fs.open(url))

In [ ]:
type(refs)

In [ ]:
%%time 
# refs is original references JSON dict
make_parquet_store('refs_test', refs, compression='zstd')

In [12]:
%%time
mapper = ParquetReferenceMapper('refs_test')
fs = fsspec.filesystem('reference', fo=mapper, remote_protocol='s3', remote_options=dict(anon=True))
ds = xr.open_dataset(fs.get_mapper(''), engine='zarr')

CPU times: user 28.6 ms, sys: 7 ms, total: 35.6 ms
Wall time: 577 ms


In [13]:
%%time
da = ds.TRAD.sel(time='1990-01-01 00:00').load()

CPU times: user 487 ms, sys: 104 ms, total: 591 ms
Wall time: 1.35 s


In [14]:
%%time
da = ds.TRAD.sel(time='2015-01-01 00:00').load()

CPU times: user 387 ms, sys: 68 ms, total: 455 ms
Wall time: 766 ms


In [15]:
da.mean().data

array(266.92635398)